In [ ]:
from ultralytics import FastSAM
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
model = FastSAM("FastSAM-s.pt") #FastSAM("FastSAM-x.pt")
model.info()

In [ ]:
everything_results = model("dogs.jpg", device="cpu", retina_masks=True, imgsz=1024, conf=0.4, iou=0.9)

In [ ]:
res = everything_results[0]

In [ ]:
# Original image
img = res.orig_img.copy()

# Get the masks (N, H, W)
masks = res.masks.data.cpu().numpy()

# Create an overlay for all masks
overlay = img.copy()

# Generate a random color for each mask
for i, mask in enumerate(masks):
    color = np.random.randint(0, 255, (3,), dtype=np.uint8)
    colored_mask = np.stack([mask * c for c in color], axis=-1).astype(np.uint8)
    overlay = cv2.addWeighted(overlay, 1.0, colored_mask, 0.5, 0)

# Plot result
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Segmentation Masks (FastSAM)")
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import FastSAM

# Load model and run inference
model = FastSAM("FastSAM-x.pt")
results = model("rgb_fast_sam_test.tif", device="cpu", retina_masks=True, imgsz=512, conf=0.3, iou=0.9)

# Get first result (for one image)
res = results[0]
img = res.orig_img.copy()

# Convert to color if grayscale
if len(img.shape) == 2 or img.shape[2] == 1:
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

# Get masks
masks = res.masks.data.cpu().numpy()

# Draw contours on a copy of the original image
outline_img = img.copy()

for mask in masks:
    mask_uint8 = (mask * 255).astype(np.uint8)
    contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    color = tuple(np.random.randint(0, 255, size=3).tolist())
    cv2.drawContours(outline_img, contours, -1, color, thickness=2)

# Plot
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(outline_img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Segmentation Outlines (FastSAM)")
plt.show()


In [ ]:
# Test Mobile SAM?

from ultralytics.models.sam import Predictor as SAMPredictor

overrides = dict(task="segment", mode="predict", imgsz=1024, model="mobile_sam.pt")
predictor = SAMPredictor(overrides=overrides)

predictor.set_image("rgb_fast_sam_test.tif")

results = predictor()

# Get first result (for one image)
res = results[0]
img = res.orig_img.copy()

# Convert to color if grayscale
if len(img.shape) == 2 or img.shape[2] == 1:
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

# Get masks
masks = res.masks.data.cpu().numpy()

# Draw contours on a copy of the original image
outline_img = img.copy()

for mask in masks:
    mask_uint8 = (mask * 255).astype(np.uint8)
    contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    color = tuple(np.random.randint(0, 255, size=3).tolist())
    cv2.drawContours(outline_img, contours, -1, color, thickness=2)

# Plot
plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(outline_img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Segmentation Outlines (FastSAM)")
plt.show()


In [ ]:
help(SAM.predict)

In [ ]:
import leafmap

m = leafmap.Map(center=[56.02437988, -2.76394275], zoom=17)
#m.add_basemap("SATELLITE")
#m.layers[-1].visible = False
m.add_raster("test_s2_3bands.tif", layer_name="Image")
m